In [61]:
import sys
sys.path.append('/vera/u/olwitt/illustris_python/illustris_python')
from loadMPBs import loadMPBs

import time
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import h5py
import numba as nb
from numba import jit, njit
import tracerFuncs as tF
import insituFuncs as iF

plt.style.use('default')
plt.rcParams["figure.figsize"][0] = 16
plt.rcParams["figure.figsize"][1] = 9
plt.rcParams['errorbar.capsize']=2

In [3]:
basePath='/virgotng/universe/IllustrisTNG/TNG50-4/output'
h_const=il.groupcat.loadHeader(basePath,99)['HubbleParam']
boxSize = il.groupcat.loadHeader(basePath,99)['BoxSize']
print(h_const)

0.6774


# where is tracer parent?

find out, where the tracer parent sits, i.e. whether it's still in the main progenitor or in another galaxy or in the IGM

In [98]:
def location_of_parents(parent_index_filename, basePath, start_snap, target_snap):
    """first output returns the subhalo index if the tracer parent particle sits in a galaxy or -1 if it's in the IGM
    second output states, whether parent particle is inside main progenitor"""
    start = time.time()
    #load file
    index_file = h5py.File(parent_index_filename,'r')
    parent_indices = index_file['parent_indices'][:,:]
    
    location = np.empty(parent_indices[:,0].shape,dtype=int)
    location.fill(-1)
    
    f = h5py.File(basePath[:-6] + 'postprocessing/offsets/offsets_0' + str(target_snap) + '.hdf5','r')
    starsInSubOffset = f['Subhalo/SnapByType'][:,4]
    gasInSubOffset = f['Subhalo/SnapByType'][:,0]
    
    
    #load number of particles per galaxy to avoid classifying parents as bound to a galaxy 
    #while they're in reality "inner fuzz"
    #of a halo
    num_in_sub = il.groupcat.loadSubhalos(basePath, target_snap, fields = ['SubhaloLenType'])
    gas_num_in_sub = num_in_sub[:,0].copy()
    star_num_in_sub = num_in_sub[:,4].copy()
    del num_in_sub
    end_load = time.time()
    print('time for loading: ',end_load-start)
    
    #find parent index in offset files
    which = parent_indices[:,1]
    indices = parent_indices[:,0]     
    for i in nb.prange(parent_indices[:,0].shape[0]):
        if which[i] == 0: #gas parent
            ind = np.searchsorted(gasInSubOffset,indices[i],'right')
            ind-=1 #compensate for how np.searchsorted chooses the index
            
            #only important for last galaxy of halo: check whether particle is inner fuzz, i.e. IGM particles
            if gasInSubOffset[ind] + gas_num_in_sub[ind] < indices[i]:
                continue
            
        else: #star parent
            ind = np.searchsorted(starsInSubOffset,indices[i],'right')
            ind-=1 #compensate for how np.searchsorted chooses the index
            
            #only important for last galaxy of halo: check whether particle is inner fuzz, i.e. IGM particles
            if starsInSubOffset[ind] + star_num_in_sub[ind] < indices[i]:
                continue
                
        location[i]=ind
    
    end_loc = time.time()
    print('time for locating: ',end_loc - end_load)
    
    #now identify parents that are still in their (main progenitor) galaxy
    g = h5py.File(basePath[:-6] + 'postprocessing/offsets/offsets_0' + str(start_snap) + '.hdf5','r')
    starsInSubOffset_start_snap = g['Subhalo/SnapByType'][:,4]
    stars_in_subs_start_snap = il.groupcat.loadSubhalos(basePath, start_snap, fields = ['SubhaloLenType'])[:,4]
    
    #########################################problem with fuzz???##########################################
    #calculate _inSitu_ star offsets again
    check = h5py.File(basePath[:-6] + 'postprocessing/StellarAssembly/stars_0' + str(start_snap) + '.hdf5','r')
    insitu = np.asarray(check['InSitu'][:]==1)
    check.close()
    insituStarsInSubOffset = np.zeros(starsInSubOffset_start_snap.shape[0])
    for i in range(1,starsInSubOffset_start_snap.shape[0]):
        star_indices = np.arange(starsInSubOffset_start_snap[i-1],starsInSubOffset_start_snap[i])
        insitu_indices = insitu[star_indices]
        insituStarsInSubOffset[i] = len(np.nonzero(insitu_indices)[0])
    insituStarsInSubOffset = np.cumsum(insituStarsInSubOffset)
    tracersInParentsOffsets = index_file['tracers_in_parents_offset'][:].copy()
    parentsInSubOffset = tF.tracersInSubhalo(insituStarsInSubOffset,tracersInParentsOffsets).astype(int)
    ######################################################################################################
    end_insitu = time.time()
    print('time for insitu offsets: ',end_insitu - end_loc)
    
    numSubs = il.groupcat.loadSubhalos(basePath,99,fields=['SubhaloPos']).shape
    ids = np.arange(numSubs[0])

    #load all MPBs
    tree_ids = loadMPBs(basePath,99,ids=ids, fields=['SubfindID'])
    end_loadMPBs = time.time()
    print('time for loading MPBs: ',end_loadMPBs - end_loc)
    
    isInMP = np.empty(parent_indices[:,0].shape,dtype=bool)
    isInMP.fill(False)
    tree_check = list(tree_ids)
    end_check = time.time()
    print('time for checking trees: ',end_check - end_loadMPBs)
    for j in nb.prange(1,parentsInSubOffset.shape[0]): #loop over all galaxies at z=0
        #find all associated particles:
        ParentIndicesInSub = np.arange(parentsInSubOffset[j-1],parentsInSubOffset[j])
        
        if j not in tree_check: #if subhalo hast no tree, skip it and assign "False"
            continue
        if tree_ids[j]['SubfindID'].shape[0] < start_snap - target_snap: #if tree doesn't reach until target_snap
            continue
        main_prog = tree_ids[j]['SubfindID'][start_snap - target_snap]
        isInMP[np.where(location[ParentIndicesInSub] == main_prog)[0]] = True
        
    end = time.time()
    print('time for checking MPs: ',end - end_check)
    
    f.close()
    g.close()
    index_file.close()
    return location, isInMP

In [99]:
location, isInMP = location_of_parents('files/TNG50-4/parent_indices_redshift_0.0.hdf5',\
                                       basePath, start_snap = 99, target_snap = 99)

time for loading:  0.023464679718017578
time for locating:  6.0743584632873535
time for insitu offsets:  0.10866904258728027
time for loading MPBs:  1.6822624206542969
time for checking trees:  0.0007262229919433594
time for checking MPs:  45.67642021179199


In [101]:
print(np.where(isInMP==True)[0].shape)

(3361,)
